<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/revisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install as libraries necessárias

In [ ]:
!pip install pycaret

In [ ]:
!pip install shap

### Carregar as libraries

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt # Gráficos

import seaborn as sns
sns.set_theme(style = 'ticks', palette = 'pastel')

from pycaret import regression, classification # Canivete suiço para Machine Learning
import shap

from collections import Counter

import tensorflow as tf
tf.__version__

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

### Configurações de ambiente

In [ ]:
pd_options = pd.options.display # Objeto para configuração do ambiente: número de linhas, colunas, precisão e etc

In [ ]:
pd_options.max_columns = 50 # número máximo de colunas
pd_options.max_rows = 50 # número máximo de linhas
pd_options.precision = 3 # número de casas decimais

### Facets - DataViz

In [ ]:
from IPython.core.display import display, HTML
import base64

In [ ]:
!pip install facets-overview==1.0.0

In [ ]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

### Carregar os dados de treinamento e teste

In [ ]:
url_T = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/train_LABDATA.csv';
url_V = 'https://raw.githubusercontent.com/MathMachado/DataFrames/master/test_LABDATA.csv';

In [ ]:
df_T = pd.read_csv(url_T, index_col = 'id');
df_V = pd.read_csv(url_V, index_col = 'id');

In [ ]:
df_T.head()

### Tipos dos dados

In [ ]:
df_T.dtypes

### Conversão de tipos
#### TotalCharges
* A variável TotalCharges é do tipo 'object'. Entretanto, esta variável deveria ser do tipo 'float'.         

In [ ]:
df_T['TotalCharges3'] = pd.to_numeric(df_T['TotalCharges'])

dado que temos valores ' ' na variável TotalCharges, então precisamos usar errors = 'coerce'. Vamos olhar para a linha 161 que inviabiliza o uso do pd.astype(np.float):

In [ ]:
df_T.iloc[161]

In [ ]:
df_T['TotalCharges2'] = pd.to_numeric(df_T['TotalCharges'], errors = 'coerce')
df_T.dtypes

In [ ]:
df_T[['TotalCharges2', 'TotalCharges']].head(50)

In [ ]:
df_T.iloc[161]

Me parece que a conversão foi bem-sucedida.

Next: deletar a variável original.

In [ ]:
df_T = df_T.drop(columns = ['TotalCharges'], axis = 1)

In [ ]:
df_T.dtypes

#### SeniorCitizen
* A variável SeniorCitizen é do tipo 'int64'. Entretanto, esta variável deveria ser do tipo categórica.

In [ ]:
Counter(df_T['SeniorCitizen'])

In [ ]:
d_SeniorCitizen = {0 : 'No', 1: 'Yes'}

In [ ]:
df_T['SeniorCitizen2'] = df_T['SeniorCitizen'].map(d_SeniorCitizen)
Counter(df_T['SeniorCitizen2'])

Alternativamente, poderíamos ter usado objeto.value_counts():

In [ ]:
df_T['SeniorCitizen2'].value_counts()

In [ ]:
df_T = df_T.drop(columns = ['SeniorCitizen'], axis = 1)

A seguir, listas por tipos de variáveis:

In [ ]:
l_features_numericas = list(df_T.select_dtypes([np.float64, np.int64]))
l_features_numericas.remove('Churn')
l_features_numericas

In [ ]:
l_features_categoricas = list(df_T.select_dtypes(np.object))
l_features_categoricas

### DataViz com Facets

In [ ]:
def mostra_dados(df):
    fsg = FeatureStatisticsGenerator()
    dataframes = [{'table': df, 'name': 'dados de treinamento'}]

    censusProto = fsg.ProtoFromDataFrames(dataframes)
    protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")

    HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
    html = HTML_TEMPLATE.format(protostr=protostr)
    display(HTML(html))

In [ ]:
mostra_dados(df_T)

### Tratamento dos Missing Values
* Como vamos tratar os missing values?
    * **Proposição**: Construir modelos preditivos para estimar os valores missing em função das demais variáveis que NÃO SÃO missing.
    * Variáveis:
        * **Numéricas**:
            * TotalCharges2 (0.14% de missing values);
            * tenure (8.18% de missing values).
        * **Categóricas**:
            * PaymentMethod (1.76% de missing values);
            * Dependents (3.87% de missing values);

A seguir, funções para nos ajudar a construir os modelos de Machine Learning para estimar os missing values:

In [ ]:
def prepara_dataframes(df, target, l_colunas_mv):
    df2 = 'df' + '_' + target + '_sem_mv'
    #print(df2)

    l_colunas_mv.remove(target)
    print(f'Features ignoradas: {l_colunas_mv}')
    df2 = df.copy()
    df3 = df.copy()

    df2 = df2.dropna() # Excluir todas as linhas com missing values
    df2 = df2.drop(columns = ['Churn'], axis = 1)

    # Para reduzir o viés, vamos dropar/deletar as features que são missing values
    df2 = df2.drop(columns = l_colunas_mv, axis = 1)
    print(df2.isna().sum())

    # Apontar os missing values no dataframe original:
    df3[target+'_mv'] = np.where(df3[target].isna(), 1, 0)

    return (df2, df3)

In [ ]:
l_colunas_mv = ['TotalCharges2', 'tenure', 'PaymentMethod', 'Dependents']

In [ ]:
df_T_copia = df_T.copy()

In [ ]:
df_TotalCharges2_sem_mv, df_T = prepara_dataframes(df_T, 'TotalCharges2', l_colunas_mv)

In [ ]:
df_T.head()

In [ ]:
l_colunas_mv = ['TotalCharges2', 'tenure', 'PaymentMethod', 'Dependents']

for target in l_colunas_mv:
    df_temp, df_T = prepara_dataframes(df_T, target, l_colunas_mv)
    vars()[f"df_{target}_sem_mv"] = df_temp.copy()

In [ ]:
df_tenure_sem_mv.head()

In [ ]:
df_temp.head()

In [ ]:
df_TotalCharges2_sem_mv
df_tenure_sem_mv
df_PaymentMethod_sem_mv
df_Dependents_sem_mv

In [ ]:
df_tenure_sem_mv, df_T = prepara_dataframes(df_T, 'tenure', l_colunas_mv)